In [1]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
from easymore.easymore import easymore
import os



/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [ ]:
df = pd.read_csv('../lake_param/lake_param_HYPE.csv')


df = df [df['CTR_LONWGS84']>-120]
df = df [df['CTR_LONWGS84']<-105]
df = df [df['CTR_LATWGS84']>47]
df = df [df['CTR_LATWGS84']<53]

print(df)

df.to_csv('../lake_param/lake_param_HYPE_slice.csv')


## match the grand id and lake id for the lake_param_HYPE_slice and save in lake_param_HYPE_grand_to_HydroLake

In [51]:
if os.path.isfile('../lake_param/lake_param_HYPE_slice.csv'):
    os.remove('../lake_param/lake_param_HYPE_slice.csv')
df = pd.read_csv('../lake_param/lake_param_HYPE.csv')
link = pd.read_csv('../lake_param/lake_param_HYPE_grand_to_HydroLake.csv')

Grand_id = np.array(link['GRAND_ID'])
Hylak_id = np.array(link['Hylak_id'])

indx = df.index[df['GRAND_ID'].isin(Grand_id)]
print(indx)

df['Hylak_id'] = 0 # set the existing lakes to zero
df['Hylak_id'].loc[indx] = Hylak_id # pass the hylak_id

# remove the Hylak_id that are 0
df = df[df['Hylak_id']>0]
df = df.reset_index(drop=True)

df.to_csv('../lake_param/lake_param_HYPE_Saskatoon.csv')


Int64Index([6, 18, 69, 86, 94, 111, 114, 119, 126, 142, 143], dtype='int64')


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [61]:
Ntopo = pd.read_csv('../network_topology/Saskatoon.csv')
Hanasaki = pd.read_csv('../lake_param/lake_param_Hanasaki_Saskatoon.csv')
HYPE = pd.read_csv('../lake_param/lake_param_HYPE_Saskatoon.csv')

Ntopo_before = len(Ntopo)



Hanasaki['ID'] = Hanasaki['lakeId']
HYPE['ID'] = HYPE['Hylak_id']

HYPE_before = print(len(HYPE))

# if there is a HYPE and Hanasaki choose Hanasaki
shared_ID = np.intersect1d(np.array(Hanasaki['ID']), np.array(HYPE['ID']))
if shared_ID.size == 0:
    HYPE_retain = np.setdiff1d(np.array(HYPE['ID']), shared_ID)
    HYPE = HYPE [HYPE['ID'].isin(HYPE_retain)]

HYPE_after = print(len(HYPE))

if HYPE_before != HYPE_after:
    print('the IDs were removed from the HYPE, to be considered as Hanasaki')
    print(shared_ID)
    if len(shared_ID) == (HYPE_before - HYPE_after):
        print('number of HYPE reduced:', HYPE_before - HYPE_after)
        print('number of shared_ID', len(shared_ID))
    else:
        print('sth is wrong!')


Ntopo = pd.merge(Ntopo, Hanasaki, on=['ID'], how='outer')
Ntopo = pd.merge(Ntopo, HYPE, on=['ID'], how='outer')


Ntopo_after = len(Ntopo)

Ntopo = Ntopo.fillna(-9999)

if Ntopo_before == Ntopo_after:
    print('Ntopo before and after are the same length, seems good to go!')
    Ntopo.to_csv('../lake_param/test.csv')
else:
    print('sth is wrong, Ntopo before and after are different')


11
11
Ntopo before and after are the same length, seems good to go!


In [62]:
# save the network topology
Ntopo = pd.read_csv('../network_topology/Saskatoon_NTOPO_lake.csv')
Ntopo_flags = pd.read_csv('../network_topology/Saskatoon_NTOPO_lake_flag.csv')
Ntopo_file_name = '../network_topology/Saskatoon_NTOPO_lake.nc'
nc_creation(Ntopo, Ntopo_flags, Ntopo_file_name)

#
ds = xr.open_dataset('../network_topology/Saskatoon_NTOPO_lake.nc')
print(ds)


,Unnamed: 0_x,ID,next_ID,area,length,is_lake,outflow,inflow,endorheic,slope,...,BR_MX_MAIN,BR_MN_MAIN,BR_MX_BR,BR_DEST_LONWGS84,BR_DEST_LATWGS84,SOURCE,SOURCE_ELEVS,SOURCE_OPS,SOURCE_SPILL,SOURCE_BRAN
0,0,643,71027284,3.917200e+08,1.000000,1,-1.0,-1.0,0,0.001,...,0.0,0.0,0.0,0.0,0.0,11.0,HHYPEv3 DamData,HHYPEv3 DamData,AHYPEv4 LakeData,*NONE*
1,1,7191,71023817,7.928000e+07,1.000000,1,-1.0,-1.0,0,0.001,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999
2,2,7209,71022872,8.619000e+07,1.000000,1,-1.0,-1.0,0,0.001,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999
3,3,7237,71023994,4.238000e+07,1.000000,1,-1.0,-1.0,0,0.001,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999
4,4,7363,71029837,1.207000e+07,1.000000,1,-1.0,-1.0,0,0.001,...,0.0,0.0,0.0,0.0,0.0,11.0,HHYPEv3 DamData,HHYPEv3 DamData,AHYPEv4 LakeData,*NONE*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3058,3058,71046848,71045374,3.918906e+07,5247.541353,0,-1.0,-1.0,0,0.001,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999
3059,3059,71046869,71045147,7.584365e+07,14100.790971,0,-1.0,-1.0,0,0.001,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999
3060,3060,71046901,71044780,3.094328e+07,2285.562412,0,-1.0,-1.0,0,0.001,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999
3061,3061,71046927,71045205,3.686062e+07,5105.747478,0,-1.0,-1.0,0,0.001,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999
